# Lexical Search 실습

### 목차

- **[준비](#준비)**
  1. [Collection 생성](#1.-Collection-생성)
  2. [Dashboard 접속](#1.-Dashboard-접속)
- **[실습](#실습)**
  1. [기본 설정 (nori 미사용)](#1.-기본-설정-(nori-미사용))
  2. [nori 미사용 시 한글 검색의 문제점](#2.-nori-미사용-시-한글-검색의-문제점)
  3. [nori 형태소 분석기 적용](#3.-nori-형태소-분석기-적용)
  4. [nori 사용 시 한글 검색 개선 확인](#4.-nori-사용-시-한글-검색-개선-확인)
  5. [nori 사용 시에도 발생하는 한계점 탐색](#5.-nori-사용-시에도-발생하는-한계점-탐색)
  6. [동의어 사전 사용](#6.-동의어-사전-사용)
- **[결론](#결론)**

## 준비

### 1. Collection 생성

- AWS 콘솔에서 [OpenSearch](https://us-west-2.console.aws.amazon.com/aos/home?region=us-west-2#opensearch/collections)로 이동
- 좌측 네비게이션 메뉴에서 **`Serverless > Collections`** 으로 이동
- **Create collection** 버튼 클릭
  - Collection 이름: lexical-search
  - Collection 타입: Search
  - Security: Easy create
  - 나머지는 기본값으로 유지하고 **Next** 클릭
  - 내용을 리뷰하고 **Submit** 클릭
- 약 3분가량 기다리면 생성 완료

### 2. Dashboard 접속

- Collection 목록에서 방금 생성한 lexical-search 클릭
- **Manage network access** 클릭
- easy-lexical-search를 선택하고 **Edit** 클릭
- 아래쪽 **Enable access to OpenSearch Dashboards** 선택
- Search collection(s), or input specific prefix term(s)에서 `Collection Name = lexical-search` 추가
- **Update** 버튼 클릭
- 잠시 기다린 후, Collection 목록에서 **Dashboard** 링크를 클릭해서 Dashboard로 이동

## 실습

### 1. 기본 설정 (nori 미사용)

```es
PUT /korean-test
{
  "settings": {
    "analysis": {
      "analyzer": {
        "default": {
          "type": "standard"
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "text": {
        "type": "text"
      }
    }
  }
}
```

```
POST /korean-test/_doc
{
  "text": "오늘은 날씨가 좋아서 공원에 산책을 갔습니다."
}

POST /korean-test/_doc
{
  "text": "내일은 비가 온다고 합니다."
}
```

### 2. nori 미사용 시 한글 검색의 문제점

```
GET /korean-test/_analyze
{
  "analyzer": "standard",
  "text": "오늘은 날씨가 좋아서 공원에 산책을 갔습니다."
}
```

```
GET /korean-test/_search
{
  "query": {
    "match": {
      "text": "날씨가"
    }
  }
}
```

```
GET /korean-test/_search
{
  "query": {
    "match": {
      "text": "날씨"
    }
  }
}
```

```
GET /korean-test/_search
{
  "query": {
    "match": {
      "text": "좋은 날씨"
    }
  }
}
```

### 3. nori 형태소 분석기 적용

```
PUT /korean-test-nori
{
  "settings": {
    "analysis": {
      "analyzer": {
        "my_nori_analyzer": {
          "type": "custom",
          "tokenizer": "nori_tokenizer",
          "filter": [
            "lowercase",
            "nori_part_of_speech"
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "text": {
        "type": "text",
        "analyzer": "my_nori_analyzer"
      }
    }
  }
}
```

```
POST /korean-test-nori/_doc
{
  "text": "오늘은 날씨가 좋아서 공원에 산책을 갔습니다."
}

POST /korean-test-nori/_doc
{
  "text": "내일은 비가 온다고 합니다."
}
```

### 4. nori 사용 시 한글 검색 개선 확인

```
GET /korean-test-nori/_analyze
{
  "analyzer": "nori_analyzer",
  "text": "오늘은 날씨가 좋아서 공원에 산책을 갔습니다."
}
```

```
GET /korean-test-nori/_search
{
  "query": {
    "match": {
      "text": "날씨가"
    }
  }
}
```

```
GET /korean-test-nori/_search
{
  "query": {
    "match": {
      "text": "날씨"
    }
  }
}
```

```
GET /korean-test-nori/_search
{
  "query": {
    "match": {
      "text": "좋은 날씨"
    }
  }
}
```

### 5. nori 사용 시에도 발생하는 한계점 탐색

```
POST /korean-test-nori/_doc
{
  "text": "그는 책을 읽었다."
}

POST /korean-test-nori/_doc
{
  "text": "그녀는 독서를 했다."
}
```

```
GET /korean-test-nori/_analyze
{
  "analyzer": "nori_analyzer",
  "text": "그는 책을 읽었다."
}
```

```
GET /korean-test-nori/_analyze
{
  "analyzer": "nori_analyzer",
  "text": "그녀는 독서를 했다."
}
```

```
GET /korean-test-nori/_search
{
  "query": {
    "match": {
      "text": "독서"
    }
  }
}
```

```
GET /korean-test-nori/_search
{
  "query": {
    "match": {
      "text": "책 읽기"
    }
  }
}
```

### 6. 동의어 사전 사용

```
PUT /korean-test-improved
{
  "settings": {
    "analysis": {
      "analyzer": {
        "nori_synonym_analyzer": {
          "tokenizer": "nori_tokenizer",
          "filter": [
            "nori_part_of_speech",
            "synonym_filter"
          ]
        }
      },
      "filter": {
        "synonym_filter": {
          "type": "synonym",
          "synonyms": [
            "책 읽기, 독서, 독해"
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "text": {
        "type": "text",
        "analyzer": "nori_synonym_analyzer"
      }
    }
  }
}
```

```
POST /korean-test-improved/_doc
{
  "text": "그는 책을 읽었다."
}

POST /korean-test-improved/_doc
{
  "text": "그녀는 독서를 했다."
}
```

```
GET /korean-test-improved/_search
{
  "query": {
    "match": {
      "text": "책 읽기"
    }
  }
}
```

## 결론

지금까지 한글 형태소 분석기를 활용해서 한글 검색 성능을 향상시키는 방법을 알아봤습니다. 

우리가 사용할 수 있는 검색 성능을 향상시킬 수 있는 방법은 매우 다양합니다. 예를 들어 원본 텍스트와 함께 정규화된 형태의 필드를 추가하여 **다중 필터를 사용**함으로써 검색의 유연성을 높일 수 있습니다. 그리고, **n-gram 토큰화**를 사용하면 부분 일치를 허용할 수 있습니다. 이 외에도 **퍼지 검색**을 사용하여 유사한 단어를 허용할 수 있습니다. 

각 방법은 장단점이 있으므로, 실제 데이터와 요구사항에 맞게 적절히 선택하고 조정해야 합니다.